In [1]:
import os
import glob
import pandas as pd
from datetime import datetime
import time

In [2]:
variable_path_base = r'../data/variables'
variable_files_pattern = f'var-dia-*-raw.zip'
variable_datetime_format = '%Y-%m-%d' #'%Y-%m-%d'
variable_column_Valor = 'Valor' 
variable_file_output = 'var-mes-{etiqueta}-raw.csv'

In [3]:
variable_path_base = variable_path_base.split('/')
variable_files = glob.glob(os.path.join(*variable_path_base, variable_files_pattern))
print(len(variable_files), variable_files)

7 ['..\\data\\variables\\var-dia-PTPM_CON-raw.zip', '..\\data\\variables\\var-dia-PT_10_TT_D-raw.zip', '..\\data\\variables\\var-dia-Q_MEDIA_D-raw.zip', '..\\data\\variables\\var-dia-Q_MN_D-raw.zip', '..\\data\\variables\\var-dia-Q_MX_D-raw.zip', '..\\data\\variables\\var-dia-TMN_CON-raw.zip', '..\\data\\variables\\var-dia-TMX_CON-raw.zip']


In [4]:
dtypes = {
    'CodigoEstacion':'string','Etiqueta':'category'#,'Fecha': 'string',
    #'IdParametro':'category','DescripcionSerie':'category','Frecuencia':'category','Grado':'category','Calificador':'category','NivelAprobacion':'category',
}

usecols = {
    'CodigoEstacion','Etiqueta', 'Fecha', variable_column_Valor
    #'IdParametro','DescripcionSerie','Frecuencia','Grado','Calificador','NivelAprobacion'
}

dateparse = lambda x: datetime.strptime(x, variable_datetime_format)


In [5]:
def groupVariableByMonth(f):
    start_time = time.time()
    
    print(f'Archivo: {f}')
    print(f'Leyendo h:{time.asctime()}...')
    df = pd.read_csv(f, dtype = dtypes, parse_dates = ['Fecha'], usecols = usecols, date_parser = dateparse)#, index_col = ['Fecha', 'CodigoEstacion'])
    print(f'**** r:{df.shape} h:{time.asctime()} t:{(time.time() - start_time) / 60} ****')
    
    df.rename(columns = {variable_column_Valor: 'Valor'}, inplace = True)
    
    etiqueta = df.loc[0,'Etiqueta']
    
    print(f'Etiqueta leída: {etiqueta}')
    
    print(f'Agrupando por mes...')
    dfm = df.groupby([df.CodigoEstacion,df.Etiqueta,df.Fecha.dt.to_period('M')]).Valor.agg(['min','mean','median','max']).dropna().reset_index()
    print(f'**** r:{dfm.shape} h:{time.asctime()} t:{(time.time() - start_time) / 60} ****')
    print(dfm.head())
    
    p = os.path.join(*variable_path_base, variable_file_output.format(etiqueta = etiqueta))
    
    print(f'Guardando en {p}...')
    dfm.to_csv(p, index = False)
    print(f'**** h:{time.asctime()} t:{(time.time() - start_time) / 60} ****')

In [6]:
%%time
for f in variable_files:
    groupVariableByMonth(f)


Archivo: ..\data\variables\var-dia-PTPM_CON-raw.zip
Leyendo h:Fri Jun 19 09:56:20 2020...


KeyboardInterrupt: 

In [7]:
#estacion = '13067020'
#year = 2005
#print(dfm.loc[(dfm.CodigoEstacion == estacion) & (dfm.Fecha.dt.year == year)].head(2))
#print(df[(df.CodigoEstacion == estacion) & (df.Fecha.dt.year == year)].head(2))